In [2]:
install.packages('neuralnet')


The downloaded binary packages are in
	/var/folders/qk/0vhw5p4n2610vyrs2zqjmj2h0000gn/T//RtmpELQOBZ/downloaded_packages


In [3]:
library(neuralnet)

In [4]:
train_data = read.csv('train.csv')
test_data = read.csv('test.csv')

In [5]:
target=test_data$target
cnt=test_data$cnt
target_max<-max(train_data$target)
target_min<-min(train_data$target)

In [7]:
de_normalization<-function(data,train_max,train_min){
  return(as.integer(data*(train_max-train_min)+train_min))
}

In [8]:
n<-names(train_data)
use<-c("temp","atemp","hum", "windspeed" ,"cnt_normal","week_moving_avg_normal",
       "season_1","season_2","season_3","season_4",
       "mnth_1","mnth_2","mnth_3","mnth_4","mnth_5","mnth_6","mnth_7","mnth_8","mnth_9","mnth_10","mnth_11","mnth_12",
       "holiday_1","holiday_2",
       "weekday_1","weekday_2","weekday_3","weekday_4","weekday_5","weekday_6","weekday_7",
       "workingday_1","workingday_2",
       "weathersit_1","weathersit_2","weathersit_3"
)

In [9]:
f<-as.formula(paste("target_normal~",paste(n[n %in% use],collapse = '+')))

In [10]:
# all var. in bp network
nn<-neuralnet(f,data=train_data,hidden =5,linear.output = TRUE)
test_data<-test_data[,use]
print(nn)
plot(nn)
results<-compute(nn,test_data)
res<-results$net.result

$call
neuralnet(formula = f, data = train_data, hidden = 5, linear.output = TRUE)

$response
    target_normal
1   0.09408410549
2   0.06967213115
3   0.15858873842
4   0.14825374198
5   0.13025659301
6   0.17373485388
7   0.17640769779
8   0.14558089808
9   0.13774055595
10  0.10138987883
11  0.04490377762
12  0.21721311475
13  0.26657163222
14  0.19814682823
15  0.09800427655
16  0.09889522452
17  0.17551674982
18  0.27690662865
19  0.01336421953
20  0.00000000000
21  0.13114754098
22  0.11885245902
23  0.11849607983
24  0.19066286529
25  0.16553813257
26  0.19511760513
27  0.19939415538
28  0.22754811119
29  0.10228082680
30  0.21240199572
31  0.22826086957
32  0.19583036351
33  0.20919458304
34  0.19725588026
35  0.23431931575
36  0.18549536707
37  0.20634354954
38  0.26407697790
39  0.24661439772
40  0.30007127584
41  0.36421952958
42  0.44476122594
43  0.21454027085
44  0.24607982894
45  0.12045616536
46  0.18157519601
47  0.26478973628
48  0.24518888097
49  0.18353528154
50  0.2

In [11]:
res1<-de_normalization(res,target_max,target_min)


final_tb_all<-cbind(res1,test_data,target,cnt)

In [12]:
#_-------tuning by domain------
for(i in 1:nrow(final_tb_all)){
  
  if(final_tb_all$res1[i]<=0){
    final_tb_all$res1[i]=final_tb_all$cnt[i]
  }
  
}
#-----------yanzheng-------
profit <- sum(pmin(final_tb_all$target,final_tb_all$res1)*3 - final_tb_all$res1*2)
profit_benchmark<-sum(pmin(final_tb_all$target,final_tb_all$cnt)*3-final_tb_all$cnt*2)

In [13]:
profit_benchmark

[1] 1442972